# Perceptrón aplicado a iris

**Lectura del corpus:** $\;$ cargamos el corpus Iris y comprobamos que las matrices de datos `X` y etiquetas `y` contienen el número de filas y columnas que esperamos.

In [1]:
import numpy as np;
from sklearn.datasets import load_iris;
iris = load_iris(); 
X = iris.data.astype(np.float16);
y = iris.target.astype(np.uint).reshape(-1, 1);
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

(150, 4) (150, 1) 
 [[5.1015625  3.5        1.40039062 0.19995117 0.        ]
 [4.8984375  3.         1.40039062 0.19995117 0.        ]
 [4.69921875 3.19921875 1.29980469 0.19995117 0.        ]
 [4.6015625  3.09960938 1.5        0.19995117 0.        ]
 [5.         3.59960938 1.40039062 0.19995117 0.        ]]


**Partición del corpus:** $\;$ Creamos un split del dataset iris con un $20\%$ de datos (30 muestras) para evaluación (*test*), y el resto (120 muestras) para entrenamiento (*training*), barajando previamente los datos de acuerdo con una semilla dada para la generación de números aleatorios. Aquí, como en todo código que incluya aleatoriedad (que requiera generar números aleatorios), conviene fijar dicha semilla para poder reproducir experimentos posteriormente con exactitud. En este caso, usaremos como semilla un objeto `int` con valor 23.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
       train_test_split(X, y, test_size=0.2, 
                        shuffle=True, 
                        random_state=23)
print(X_train.shape, X_test.shape)

(120, 4) (30, 4)


**Implementación de Perceptrón:** $\;$ aplica el algoritmo de aprendizaje perceptrón, partiendo de pesos $\mathbf{W}$ nulos. 

Recibe como parámetros de entrada:

- Matriz de muestras de entrenamiento `X`,
- Vector de etiquetas de clase `y`,
- Valores de los hiperparámetros del algoritmo (a optimizar de manera experimental):
    - Variable de margen `b` $\ge 0$,
    - Factor de aprendizaje `a` $> 0$,
    - Número máximo de iteraciones `K` $> 0$. 

Devuelve:

- Pesos optimizados `W`, en notación homogénea, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C}\;$,  
- Número de muestras de train incorrectamente clasificadas `E` durante la última iteración realizada,
- Número de iteraciones ejecutadas `k`.

In [8]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; 
    Y = np.unique(y); 
    C = Y.size; 
    W = np.zeros((1+D, C));
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]]);
            cn = np.squeeze(np.where(Y==y[n]));
            gn = W[:,cn].T @ xn; 
            err = False;
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; 
                    err = True;
            if err:
                W[:, cn] = W[:, cn] + a*xn; 
                E = E + 1;
        if E == 0:
            break;
    return W, E, k

**Aprendizaje de un clasificador (lineal) con Perceptrón:** $\;$ Perceptrón minimiza el número de errores de entrenamiento (con margen `b`)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{Y}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [9]:
W, E, k = perceptron(X_train, y_train);
print("Número de iteraciones ejecutadas: ", k);
print("Número de errores de entrenamiento durante la última iteración: ", E);
print("Vectores de pesos de las clases (en columnas y en notación homogénea):\n", W);

Número de iteraciones ejecutadas:  200
Número de errores de entrenamiento durante la última iteración:  2
Vectores de pesos de las clases (en columnas y en notación homogénea):
 [[  10.           85.         -142.        ]
 [ -49.421875    -68.19140625 -176.47265625]
 [  50.171875     -1.72460938 -181.06445312]
 [-189.91210938  -87.70507812   68.69726562]
 [ -86.40258789 -137.78149414  157.88415527]]


**Cálculo de la tasa de error en test:**

In [10]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Tasa de error en test: {err_test:.1%}")

Tasa de error en test: 16.7%


**Interpretación de resultados:** $\;$ los datos de entrenamiento no parecen linealmente separables; con margen $b=0.1$ se obtiene un error de clasificación en test del $16.7\%$.

**Ajuste del margen:** $\;$ experimento para optimizar el valor del hiperparámetro $b$

In [27]:
print("# {:>6s}\t{:>2s}\t{:>4s}".format("b", "E", "k"));
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)
    print("  {:6.2f}\t{:>2d}\t{:>4d}".format(b, E, k));

#      b	 E	   k
    0.00	 3	1000
    0.01	 5	1000
    0.10	 3	1000
   10.00	 6	1000
  100.00	 6	1000


Viendo esta tabla (número de errores en entrenamiento durante la última iteración del algoritmo perceptrón en función de `b`) resulta dificil extraer conclusiones. Deberíamos evaluar la tasa de error en test para determinar el valor de `b` óptimo (que permite generalizar mejor).

**Ejercicio**: modifica la celda de código anterior para que calcule la tasa de error en test para cada experimento y lo muestre en la tabla en una nueva columna. 